In [1]:
import cv2
import numpy as np
import os
from PIL import Image
import time

In [2]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame', frame)
    cv2.imshow('gray', gray)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [8]:
import cv2
import numpy as np
import os
from PIL import Image
import requests

url = 'http://192.168.43.32:8080/'

cam = cv2.VideoCapture(url)

while True:
    ret, frame = cam.read()
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame', frame)
    #cv2.imshow('gray', gray)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

In [13]:
# Detection of Faces

faceCascade = cv2.CascadeClassifier('C:\\Users\\Paras\\Anaconda3\\envs\\py36\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml')

#url = 'http://192.168.43.32:8080/'

#cam = cv2.VideoCapture(url)
cam = cv2.VideoCapture(0)

while True:
    ret, img = cam.read()
    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize = (22,22)
    )
    
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y),(x+w, y+h), (0,255,255),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        
        
    cv2.imshow('capturing', img)
    
    esc = cv2.waitKey(30) & 0xFF
    if esc == 27:
        break
cam.release()
cv2.destroyAllWindows()

In [2]:
# Capturing Faces and Labelling them
import cv2
import numpy as np
import os
from PIL import Image
import time



url = 'http://192.168.43.32:8080/'

#cam = cv2.VideoCapture(url)
cam = cv2.VideoCapture(0)
cam.set(3, 640)
cam.set(4, 480)

face_detector = cv2.CascadeClassifier('C:\\Users\\Paras\\Anaconda3\\envs\\py36\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml')

face_id = input("Enter name of person : ")
face_id = face_id.lower()

temp = os.getcwd()
path = "\\face_count\\" + face_id + ".txt"
temp = temp+path
check = os.path.exists(temp)

id = 0
count = 0

if check:
    f = open(temp, 'r')
    id = int(f.readline())
    count = int(f.readline())
    f.close()
else:
    p = os.getcwd()
    p = p + '\\face_count\\id.txt'
    id_file = open(p, 'r')
    id = int(id_file.read())
    #print(type(id))
    id_file.close()
    
    id_file = open(p, 'w')
    new_id = str(id+1)
    id_file.write(new_id)
    id_file.close()
    
    f = open(temp, 'w')
    f.write(str(id))
    count = 0
    f.close()

print("\n Starting up... \n Look at the camera and wait...")

iters = 0
while(True):
    ret, img = cam.read()
    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,255), 2)     
        iters += 1

        cv2.imwrite("face_data/User." + str(id) + '.' + face_id + '.' + 
                    str(count+iters) + ".jpg", gray[y:y+h,x:x+w])

        cv2.imshow('Face Capturing', img)

    esc = cv2.waitKey(30) & 0xFF
    if esc == 27:
        break
    elif iters >= 10:
         break
    
count += iters
f = open(temp,'w')
f.write(str(id)+'\n'+str(count))
f.close()

print("\n Exiting Trainer...")
cam.release()
cv2.destroyAllWindows()

Enter name of person : umadevi

 Starting up... 
 Look at the camera and wait...

 Exiting Trainer...


In [5]:
# Training the faces with their labels


path = 'face_data'

recognizer = cv2.face.LBPHFaceRecognizer_create()
#recognizer = cv2.face.FisherFaceRecognizer_create()

detector = cv2.CascadeClassifier("C:\\Users\\Paras\\Anaconda3\\envs\\py36\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml");

label_to_name= {}
img_numpy = []
def images_labels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L')
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        name = os.path.split(imagePath)[-1].split(".")[2]
        if id not in label_to_name:
            label_to_name[id] = name
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return faceSamples,ids

print ("\n Training faces....")
faces,ids = images_labels(path)
recognizer.train(faces, np.array(ids))

recognizer.write('trainer/trainer.yml')

print("\n {0} faces trained.".format(len(np.unique(ids))))



 Training faces....

 3 faces trained.


In [7]:
# Real time face recognizer

import serial
Arduino_Serial = serial.Serial('com12',9600)

recognizer = cv2.face.LBPHFaceRecognizer_create()
#recognizer = cv2.face.BasicFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "C:\\Users\\Paras\\Anaconda3\\envs\\py36\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_DUPLEX

id = 0

# Initialize and start realtime video capture
#url = 'http://192.168.0.3:8080/'

#cam = cv2.VideoCapture(url)
cam = cv2.VideoCapture(0)

cam.set(3, 640)
cam.set(4, 480)

minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
fps = cam.get(5)
count = 0
flag = 0

while True:
    ret, img = cam.read()
    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )
    
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,255), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

        # Check if confidence is less than 100 ==> "0" is perfect match 
        if (confidence < 60):
            name = label_to_name[id]
            confidence = "  {0}%".format(round(100 - confidence))
            count+=1
        else:
            name = ""
            confidence = "  {0}%".format(round(100 - confidence))
            count=0
        
        cv2.putText(img, str(name), (x+5,y-5), font, 1, (255,255,255), 2)
        #cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
        cv2.putText(img, str(fps), (30,30), font, 1, (255,255,255), 1)
        if name != "":
            cv2.putText(img, str(count), (100,100), font, 1, (255,255,255),2)
                    
        if count > 20:
            cv2.putText(img, "Hello "+str(name)+"!", (50,50), font, 1, (255,255,255),2)            
            Arduino_Serial.write('1')
    
    cv2.imshow('Recognizer',img) 
        
    esc = cv2.waitKey(30) & 0xFF
    if esc == 27:
        break
            
print("\n Exiting Recognizer...")
cam.release()
cv2.destroyAllWindows()

SerialException: could not open port 'com12': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [9]:
import serial
Arduino_Serial = serial.Serial('com12',9600)
Arduino_Serial.write('1')

SerialException: could not open port 'com12': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [ ]:
import serial                                 

Arduino_Serial = serial.Serial('com3',9600)  #Create Serial port object called arduinoSerialData

print (Arduino_Serial.readline())               #read the serial data and print it as line

print ("Enter 1 to ON LED and 0 to OFF LED")



while 1:                                      #infinite loop

    input_data = input()                  #waits until user enters data

    print ("you entered", input_data)           #prints the data for confirmation

    

    if (input_data == '1'):                   #if the entered data is 1 

        Arduino_Serial.write(b'1')             #send 1 to arduino

        print ("LED ON")

       

    

    if (input_data == '0'):                   #if the entered data is 0

        Arduino_Serial.write(b'0')             #send 0 to arduino 

        print ("LED OFF")

b'Connection established...\r\n'
Enter 1 to ON LED and 0 to OFF LED
1
you entered 1
LED ON
0
you entered 0
LED OFF
1
you entered 1
LED ON
1
you entered 1
LED ON
0
you entered 0
LED OFF


In [45]:
face_id = input("Enter name of person : ")
face_id = face_id.lower()

temp = os.getcwd()
path = "\\face_count\\" + face_id + ".txt"
temp = temp+path
check = os.path.exists(temp)

id = 0
count = 0

if check:
    f = open(temp, 'r')
    id = f.readline()
    count = f.readline()
    f.close()
else:
    p = os.getcwd()
    p = p + '\\face_count\\id.txt'
    id_file = open(p, 'r')
    id = int(id_file.read())
    #print(type(id))
    id_file.close()
    
    id_file = open(p, 'w')
    new_id = str(id+1)
    id_file.write(new_id)
    id_file.close()
    
    f = open(temp, 'w')
    f.write(str(id))
    count = 0
    f.close()
    
    
#print(id)
#print(count)


f = open(temp,'w')
f.write(str(id)+'\n'+str(count))
f.close()



Enter name of person : paras


In [88]:
Ipath='C:\\Users\\Paras\\Anaconda3\\envs\\py36\\face_data\\User.1.paras.27.jpg'
PIL_img = Image.open(Ipath).convert('L')
img_numpy = np.array(PIL_img,'uint8')
img_numpy

array([[24, 23, 24, ..., 31, 30, 27],
       [24, 23, 23, ..., 31, 31, 30],
       [24, 22, 21, ..., 32, 32, 33],
       ...,
       [53, 58, 62, ...,  4,  4,  5],
       [54, 60, 65, ...,  4,  4,  5],
       [54, 61, 67, ...,  4,  4,  5]], dtype=uint8)

In [2]:
from PIL import Image
import os, sys

path = "C:\\Users\\Paras\\Anaconda3\\envs\\py36\\face_data\\"
dirs = os.listdir(path)
final_size = 150;

def resize_aspect_fit():
    for item in dirs:
        if item == '.DS_Store':
            continue
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            size = im.size
            ratio = float(final_size) / max(size)
            new_image_size = tuple([int(x*ratio) for x in size])
            im = im.resize(new_image_size, Image.ANTIALIAS)
            new_im = Image.new("RGB", (final_size, final_size))
            new_im.paste(im, ((final_size-new_image_size[0])//2, (final_size-new_image_size[1])//2))
            new_im.save(f + '.jpg', 'JPEG', quality=90)
resize_aspect_fit()

In [15]:
# Detection of Faces

import cv2
import numpy as np
import os
from PIL import Image
import time
import requests

cam = cv2.VideoCapture(0)
cam.set(3, 640)
cam.set(4, 480)

face_detector = cv2.CascadeClassifier('C:\\Users\\Paras\\Anaconda3\\envs\\py36\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml')

face_id = input("Enter name of person : ")
face_id = face_id.lower()

temp = os.getcwd()
path = "\\face_count\\" + face_id + ".txt"
temp = temp+path
check = os.path.exists(temp)

id = 0
count = 0

if check:
    f = open(temp, 'r')
    id = int(f.readline())
    count = int(f.readline())
    f.close()
else:
    p = os.getcwd()
    p = p + '\\face_count\\id.txt'
    id_file = open(p, 'r')
    id = int(id_file.read())
    #print(type(id))
    id_file.close()
    
    id_file = open(p, 'w')
    new_id = str(id+1)
    id_file.write(new_id)
    id_file.close()
    
    f = open(temp, 'w')
    f.write(str(id))
    count = 0
    f.close()

print("\n Starting up... \n Look at the camera and wait...")

iters = 0
flag = 1
url = 'http://192.168.43.125:8080/shot.jpg'

while True:
    #ret, img = cap.read()
    url = 'http://192.168.43.125:8080/shot.jpg'
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    imgg = cv2.imdecode(img_arr, -1)
    img = cv2.flip(imgg, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,255), 2)     
        iters += 1

        cv2.imwrite("face_data/User." + str(id) + '.' + face_id + '.' + str(count+iters) + ".jpg", gray[y:y+h,x:x+w])

        cv2.imshow('image', img)

    esc = cv2.waitKey(30) & 0xFF
    if esc == 27:
        break
    elif iters >= 20:
         break
    
count += iters
f = open(temp,'w')
f.write(str(id)+'\n'+str(count))
f.close()

print("\n Exiting Trainer...")
cam.release()
cv2.destroyAllWindows()

Enter name of person : SAKet

 Starting up... 
 Look at the camera and wait...

 Exiting Trainer...


In [16]:
# Training the faces with their labels


path = 'face_data'

recognizer = cv2.face.LBPHFaceRecognizer_create()
#recognizer = cv2.face.FisherFaceRecognizer_create()

detector = cv2.CascadeClassifier("C:\\Users\\Paras\\Anaconda3\\envs\\py36\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml");

label_to_name= {}
img_numpy = []
def images_labels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L')
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        name = os.path.split(imagePath)[-1].split(".")[2]
        if id not in label_to_name:
            label_to_name[id] = name
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return faceSamples,ids

print ("\n Training faces....")
faces,ids = images_labels(path)
recognizer.train(faces, np.array(ids))

recognizer.write('trainer/trainer.yml')

print("\n {0} faces trained. Exiting Program...".format(len(np.unique(ids))))



 Training faces....

 2 faces trained. Exiting Program...


In [12]:
# Real time face recognizer

recognizer = cv2.face.LBPHFaceRecognizer_create()
#recognizer = cv2.face.BasicFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "C:\\Users\\Paras\\Anaconda3\\envs\\py36\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_DUPLEX

id = 0

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640)
cam.set(4, 480)

minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
fps = cam.get(5)

while True:
    ret, img = cam.read()
    #url = 'http://192.168.43.125:8080/shot.jpg'
    #img_resp = requests.get(url)
    #img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    #imgg = cv2.imdecode(img_arr, -1)
    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )

    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,255), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

        # Check if confidence is less them 100 ==> "0" is perfect match 
        if (confidence < 100):
            name = label_to_name[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            name = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(img, str(name), (x+5,y-5), font, 1, (255,255,255), 2)
        #cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
        cv2.putText(img, str(fps), (30,30), font, 1, (255,255,255), 1)
    
    cv2.imshow('camera',img) 
    
    esc = cv2.waitKey(30) & 0xFF
    if esc == 27:
        break

# Do a bit of cleanup
print("\n Exiting Recognizer...")
cam.release()
cv2.destroyAllWindows()


 Exiting Recognizer...


In [3]:
import cv2
import numpy as np
import os
from PIL import Image
import time
import requests

url = 'http://192.168.0.5:8080/jpg'
while True:
    #ret, img = cap.read()
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    
    cv2.imshow('camera',img)
    
    esc = cv2.waitKey(30) & 0xFF
    if esc == 27:
        break

KeyboardInterrupt: 

### SOCKET PROGRAMMING

In [8]:
import socket
import sys

host = '192.168.0.2'
port = 8888

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

print("Socket created")

try:
    s.bind((host,port))
except socket.error:
    print("binding failed")
    sys.exit()

print("Socket has been bounded")

s.listen(10)

print("socket is ready")

conn, addr = s.accept()
print('Connected with '+ addr[0] + ":" + str(addr[1]))
while 1:
    data = conn.recv(1024)
    print('OK.'+data.decode())
    if not data:
        break;
    conn.sendall(data)
    
conn.close()
s.close()


Socket created
binding failed


SystemExit: 

C:\Users\Paras\Anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
import cv2
import sys
 
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
 
if __name__ == '__main__' :
 
    # Set up tracker.
    # Instead of MIL, you can also use
 
    tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
    tracker_type = tracker_types[1]
 
    if int(minor_ver) < 3:
        tracker = cv2.Tracker_create(tracker_type)
    else:
        if tracker_type == 'BOOSTING':
            tracker = cv2.TrackerBoosting_create()
        if tracker_type == 'MIL':
            tracker = cv2.TrackerMIL_create()
        if tracker_type == 'KCF':
            tracker = cv2.TrackerKCF_create()
        if tracker_type == 'TLD':
            tracker = cv2.TrackerTLD_create()
        if tracker_type == 'MEDIANFLOW':
            tracker = cv2.TrackerMedianFlow_create()
        if tracker_type == 'GOTURN':
            tracker = cv2.TrackerGOTURN_create()
        if tracker_type == 'MOSSE':
            tracker = cv2.TrackerMOSSE_create()
        if tracker_type == "CSRT":
            tracker = cv2.TrackerCSRT_create()
 
    # Read video
    video = cv2.VideoCapture(0)
 
    # Exit if video not opened.
    if not video.isOpened():
        print ("Could not open video")
        sys.exit()
 
    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print ('Cannot read video file')
        sys.exit()
     
    # Define an initial bounding box
    #bbox = (287, 23, 86, 320)
 
    # Uncomment the line below to select a different bounding box
    bbox = cv2.selectROI(frame, False)
 
    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)
 
    while True:
        # Read a new frame
        ok, frame = video.read()
        frame = cv2.flip(frame, 1)
        if not ok:
            break
         
        # Start timer
        timer = cv2.getTickCount()
 
        # Update tracker
        ok, bbox = tracker.update(frame)
 
        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
 
        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
 
        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
     
        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
 
        # Display result
        cv2.imshow("Tracking", frame)
 
        # Exit if ESC pressed
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

In [1]:
# Copyright 2018 Satya Mallick (LearnOpenCV.com)

# Import modules
import cv2, sys, os

if  not (os.path.isfile('goturn.caffemodel') and os.path.isfile('goturn.prototxt')):
    errorMsg = '''
    Could not find GOTURN model in current directory.
    Please ensure goturn.caffemodel and goturn.prototxt are in the current directory
    '''

    print(errorMsg)
    sys.exit()

# Create tracker
tracker = cv2.TrackerGOTURN_create()   

# Read video
video = cv2.VideoCapture(0)

# Exit if video not opened
if not video.isOpened():
    print("Could not open video")
    sys.exit()

# Read first frame
ok,frame = video.read()
if not ok:
    print("Cannot read video file")
    sys.exit()


# Define a bounding box
bbox = (276, 23, 86, 320)

# Uncomment the line below to select a different bounding box
#bbox = cv2.selectROI(frame, False)

# Initialize tracker with first frame and bounding box
ok = tracker.init(frame,bbox)

while True:
    # Read a new frame
    ok, frame = video.read()
    frame = cv2.flip(frame, 1)
    if not ok:
        break

    # Start timer
    timer = cv2.getTickCount()

    # Update tracker
    ok, bbox = tracker.update(frame)

    # Calculate Frames per second (FPS)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);

    # Draw bounding box
    if ok:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        # Tracking failure
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

    # Display tracker type on frame
    cv2.putText(frame, "GOTURN Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);

    # Display FPS on frame
    cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

    # Display result
    cv2.imshow("Tracking", frame)
 
    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break